In [1]:
import os
import numpy as np
import pandas as pd
import re

In [2]:
cwd = '~/apps/mtc/travel-model-two/model-files/model'

In [3]:
uec = pd.read_excel(os.path.join(cwd, 'AutoOwnership.xls'), engine='xlrd', sheet_name='Auto Ownership', skiprows=3)

In [5]:
# Label front columns
labeling = {
    'Unnamed: 0': 'uec_id',
    'Unnamed: 1': 'token',
    'Unnamed: 2': 'Description',
    'Unnamed: 3': 'filter',
    'Unnamed: 4': 'formula',
    'Unnamed: 5': 'index'
}
uec = uec.rename(columns=labeling)

# Convert cells that are fully whitespace to NaN
uec = uec.replace(r'^\s*$', np.nan, regex=True)

# Drop all lines that are entirely NaN
uec = uec.dropna(how='all')

In [6]:
# Remove formulas that equal 1--it's implied
uec.loc[uec['formula'] == 1, 'formula'] = np.nan

In [7]:
 # Make sure filter and formula are strings
uec[['filter', 'formula']] = uec[['filter', 'formula']].astype(str)

In [8]:
# Convert boolean filter expressions to np.where
def bool_to_where(filt):
    # Find boolean expressions
    bool_expressions = re.findall('\@*\w+\s*[=<>]{1,2}\s*\@*\w+', filt)
    # if len(bool_expressions) == 0:
    #     return np.nan
    
    # Insert into a np.where statement
    for bool_expression in bool_expressions:
        filt = filt.replace(bool_expression, f'np.where({bool_expression}, 1, 0)')
    
    # Remove whitespace
    filt = filt.replace(' ', '')
    return filt

uec['filter'] = uec['filter'].apply(bool_to_where).tolist()

In [9]:
# Convert if formulas to np.where
def if_to_where(formula):
    if formula[:3] == 'if(':
        formula = formula.replace('if(', 'np.where(')
    formula = formula.replace(' ', '')
    return formula

# Parse if statements
uec['formula'] = uec['formula'].apply(if_to_where)

In [10]:
# Identify coefficient columns
coeff_columns = [col for col in uec.columns if col not in labeling.values()]
coeff_columns

['0_CARS',
 '1_CAR_1CV',
 '1_CAR_1AV',
 '2_CARS_2CV',
 '2_CARS_2AV',
 '2_CARS_1CV1AV',
 '3_CARS_3CV',
 '3_CARS_3AV',
 '3_CARS_2CV1AV',
 '3_CARS_1CV2AV',
 '4_CARS_4CV']

In [11]:
# Identify utilities as rows that don't have tokens (which are precalculated aliases)
utilities = uec[uec['token'].isnull()].copy()

In [12]:
def build_expression(filt, formula):
    if filt != 'nan' and formula != 'nan':
        return f'({filt}) * ({formula})'
    
    if filt != 'nan':
        return filt
     
    return formula

# Take product of filter and formula to make expressions
utilities['Expression'] = utilities.apply(lambda x: build_expression(x['filter'], x['formula']), axis=1)

In [13]:
utilities[['filter','formula','Expression']]

,filter,formula,Expression
34,nan,"np.where(hhCountyId==SFID,1,0)","np.where(hhCountyId==SFID,1,0)"
35,"np.where(drivers==1,1,0)",nan,"np.where(drivers==1,1,0)"
36,"np.where(drivers==2,1,0)",nan,"np.where(drivers==2,1,0)"
37,"np.where(drivers==3,1,0)",nan,"np.where(drivers==3,1,0)"
38,"np.where(drivers>=4,1,0)",nan,"np.where(drivers>=4,1,0)"
...,...,...,...
93,nan,%Mobility.AV.Share%==1.0,%Mobility.AV.Share%==1.0
94,nan,%Mobility.AV.Share%==0.9,%Mobility.AV.Share%==0.9
95,nan,%Mobility.AV.Share%==0.5,%Mobility.AV.Share%==0.5
96,nan,%Mobility.AV.Share%==0.1,%Mobility.AV.Share%==0.1


In [ ]:
refactor_lookup = {
    'hhCountyId': '@df.county_id',
    'SFID': 'ID_SAN_FRANCISCO',
    'drivers': 'num_drivers',
    'workers': 'num_workers',
    'youngAdults': 'num_young_adults',
    'schoolChildren': 'num_children_6_to_15',
    'youngRetirees': 'num_young_retirees',
    'oldRetirees': 'num_old_retirees',
    'HHIncome': 'income',
    'highSchoolGraduateBoolean': '#######', # Boolean indicating presence of at least one high school graduate in HH
    'detachedDwellingBoolean': '#######', # Boolean indicating detached dwelling unit (1 = mobilehome, 2 = SF detached, 3 = other) @detachedDwellingType == 2
    'intersectionDensity': '#######', # Intersection counts need to be added to land use table--not yet included
    'populationDensity': 'population_density',
    'retailDensity': '#######', # Can't find reference for how retail density is calculated in TM2.0--not included in TM1.0 land use table
    'nonMotorizedAccessibility': '#######', # Does this only come into play on a second iteration?
    'autoAccess': '#######', # Does this only come into play on a second iteration?
    'transitAccess': '#######', # Does this only come into play on a second iteration?
    'useAccessibilities': '#######', # Does this only come into play on a second iteration? **** Bool describing whether to use accessibilies ****
    'workersAutoDependency': '#######', # Does this only come into play on a second iteration?
    'studentsAutoDependency': '#######', # Does this only come into play on a second iteration?
    'workersRailProportion': '#######', # Does this only come into play on a second iteration?
    'studentsRailProportion': '#######', # Does this only come into play on a second iteration?
    'hhBoolean': '#######', # Boolean indicating that household is NOT "institutional group quarters" (i.e., dorm)
    'nonFamily': 'non_family',
    '@workAutoTime': '#######', # AV Coeff Per Hour of travel by auto for work, summed across all workers in household
    '@numPersons18to35': 'num_persons_18_to_35',
    '@numPersons65plus': 'num_persons_65_plus',
    '%Mobility.AV.Share%': '#######', # Stored in logsum.properties as a constant (0)
}

In [ ]:
def refactor(expression):
    for old, new in refactor_lookup.items():
        if old in expression:
            # If the new value is not yet known (marked as '#######')
            if bool(re.match(r'^#*$', new)):
                # Mark and return the existing value
                expression = expression.replace(old, f'##{old}##')
            else:
                # Otherwise replace old with new
                expression = expression.replace(old, new)
    return expression

utilities.Expression = utilities.Expression.apply(refactor)

In [ ]:
utilities['Label'] = ''
utilities = utilities[['Label', 'Description', 'Expression'] + coeff_columns].copy()

In [ ]:
utilities.to_csv('auto_ownership_interim.csv', index=False)

In [ ]:
# utilities.to_csv('utilities.csv')
# utilities[['filter','formula']].head()
# parse_if('if(hhCountyId == SFID, 1, 0)')

In [ ]:
households = pd.read_csv('households.csv')

In [ ]:
households.head()

In [ ]:
households.PERSONS

In [ ]:
households.eval('(PERSONS == 2) * (PERSONS/HHT)')

In [ ]:
utilities.head()

# RELOAD EXPORTED FILE WITH 'LABEL' FILLED IN

In [ ]:
utils = pd.read_csv('auto_ownership_interim.csv')
utils[coeff_columns] = utils[coeff_columns].fillna(0)

In [ ]:
utils

In [ ]:
utils.eval('"0_CARS" + "1_CAR_1CV"')

In [ ]:
# for col in coeff_columns:
#     for x,y in tuple(zip(utils['Label'], utils[col])):
#         break

In [ ]:
utils.Label = utils.Label.fillna('')

In [ ]:
coeff_values = [(x + '_O_CARS', y) for col in coeff_columns for x,y in tuple(zip(utils['Label'], utils[col]))]
coeff_values = pd.DataFrame(coeff_values, columns=['coefficient_name', 'value'])
coeff_values['constrain'] = 'F'
coeff_values.head()

In [ ]:
for col in coeff_columns:
    utils[col] = utils['Label'] + '_' + col

# utils.to_csv('auto_ownership_file.csv')
# coeff_values.to_csv('auto_ownership_coefficients.csv')
utils.head()

In [ ]:
utils